# VGG + Deep Clustering

In [1]:
# ! pip install torch torchvision numpy pydantic umap-learn interpret faiss-cpu torchsummary matplotlib pandas seaborn scikit-learn scikit-image pytorch-lightning[extra] --no-cache-dir --ignore-installed

In [1]:
import copy
import numpy as np
import os
import pickle
import torch
import torch.nn as nn
import torch.optim as optim

from data.brain_tumor_mri import BrainTumorMRI
from models.vgg_deep import DVGG
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import CSVLogger
from torchvision import transforms
from torchvision.transforms import functional as F

In [2]:
RND_SEED=31
torch.manual_seed(RND_SEED)
torch.cuda.manual_seed_all(RND_SEED)
np.random.seed(RND_SEED)

CUDA_AVAILABLE = torch.cuda.is_available()
DEVICE = torch.device("cpu:0")
BATCH_SIZE = 64
NUM_WORKERS = 4
EPOCHS = 30
DATASETS_PATH = './datasets/'
DATASET_RATIO = None

In [3]:
brain_tumor = BrainTumorMRI(DATASETS_PATH, train_size=0.7, val_size=0.3, random_state=555, transformations=transforms.Compose(
            [
                transforms.Resize((255, 255)),
                transforms.RandomHorizontalFlip(),
                transforms.RandomRotation(10),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
                ),
            ]
        ))

In [4]:
model = DVGG(dataset=brain_tumor, num_classes=4, freeze=False)

logger = CSVLogger(save_dir="./", name="experiments/dvgg", flush_logs_every_n_steps=1)
checkpoint_callback = ModelCheckpoint(
    filename="dvgg-{epoch}-{train_loss:.4f}-{train_acc:.4f}-{val_loss:.4f}-{val_acc:.4f}",
    save_top_k=3, save_last=True, monitor="val_loss", mode="min", verbose=True
)
early_stopping_callback = EarlyStopping(monitor="val_loss", patience=3, mode="min", verbose=True)

trainer = Trainer(
    max_epochs=EPOCHS,
    callbacks=[checkpoint_callback],
    log_every_n_steps=1,
    logger=logger,
    limit_train_batches=DATASET_RATIO or 1.0,
    limit_val_batches=DATASET_RATIO or 1.0,
    limit_test_batches=DATASET_RATIO or 1.0
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
`Trainer(limit_val_batches=1.0)` was configured so 100% of the batches will be used..
`Trainer(limit_test_batches=1.0)` was configured so 100% of the batches will be used..


## TRAINING

In [5]:
trainer.fit(
    model,
    train_dataloaders=brain_tumor.train_dataloader,
    val_dataloaders=brain_tumor.val_dataloader
)


  | Name      | Type               | Params
-------------------------------------------------
0 | model     | VGG                | 134 M 
1 | loss_fn   | CrossEntropyLoss   | 0     
2 | train_acc | MulticlassAccuracy | 0     
3 | val_acc   | MulticlassAccuracy | 0     
4 | test_acc  | MulticlassAccuracy | 0     
-------------------------------------------------
134 M     Trainable params
0         Non-trainable params
134 M     Total params
537.108   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Epoch 0
Feature extracting elapsed time: 979.5813 seconds.


/Users/arthuralvim/Library/Caches/pypoetry/virtualenvs/xai-clustering-b3gHD5dt-py3.9/lib/python3.9/site-packages/threadpoolctl.py:1019: RuntimeWarning: libc not found. The ctypes module in Python 3.9 is maybe too old for this OS.
  warnings.warn(


...

Validation: 0it [00:00, ?it/s]

Epoch 28, global step 3625: 'val_loss' was not in top 3


Epoch 29
Creating dataset from Clustering elapsed time: 22.2306 seconds.
Feature extracting elapsed time: 506.7209 seconds.
Clustering elapsed time: 10.1402 seconds.
Metrics: {'clu_homogeneity_score': 0.8998683600036825, 'clu_completeness_score': 0.8982309224077143, 'clu_v_measure_score': 0.8990488956401151, 'clu_adjusted_rand_score': 0.91407012801003, 'clu_adjusted_mutual_info_score': 0.8989664962742445}


Validation: 0it [00:00, ?it/s]

Epoch 29, global step 3750: 'val_loss' was not in top 3
`Trainer.fit` stopped: `max_epochs=30` reached.


In [6]:
with open(f"{trainer.logger.log_dir}/dvgg-timing.dump", "wb") as f:
    pickle.dump(model.timers, f)

In [7]:
with open(f"{trainer.logger.log_dir}/dvgg-original-targets.dump", "wb") as f:
    pickle.dump(model.original_train_targets, f)

In [14]:
print(model.model.feature_extractor)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si

## TESTING

In [8]:
batch_preds = trainer.predict(model, dataloaders=brain_tumor.test_dataloader)

Predicting: 0it [00:00, ?it/s]

In [9]:
truth_labels = []
preds_labels = []
for truth, pred in batch_preds:
    truth_labels.append(truth)
    preds_labels.append(pred)
truth_labels = np.concatenate(truth_labels)
preds_labels = np.concatenate(preds_labels)

In [10]:
with open(f"{trainer.logger.log_dir}/dvgg-preds.dump", "wb") as f:
    pickle.dump([truth_labels, preds_labels], f)